#### Gender Classification using Fingerprints
I will first start with converting those images into pixels value to extract the features from the fingerprints. Then I will split the data into training, testing and validation sets. Now let’s start with importing the libraries we need for this task and performing some steps of data preparation:


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

import os
import cv2

import matplotlib.pyplot as plt

In [ ]:
def extract_label(img_path,train = True):
  filename, _ = os.path.splitext(os.path.basename(img_path))

  subject_id, etc = filename.split('__')
  
  if train:
      gender, lr, finger, _, _ = etc.split('_')
  else:
      gender, lr, finger, _ = etc.split('_')
  
  gender = 0 if gender == 'M' else 1
  lr = 0 if lr == 'Left' else 1

  if finger == 'thumb':
      finger = 0
  elif finger == 'index':
      finger = 1
  elif finger == 'middle':
      finger = 2
  elif finger == 'ring':
      finger = 3
  elif finger == 'little':
      finger = 4
  return np.array([gender], dtype=np.uint16)

Now our next step is to load the image path to the function we created to iterate all over the images to find labels.

In [ ]:
img_size = 96

def loading_data(path,train):
    print("loading data from: ",path)
    data = []
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_resize = cv2.resize(img_array, (img_size, img_size))
            label = extract_label(os.path.join(path, img),train)
            data.append([label[0], img_resize ])
        except Exception as e:
            pass
    data
    return data

Now, the next step is to assign various directories or folders to use the loading data function on all the images:

In [ ]:
Real_path = "../input/aman/web/Real"
Easy_path = "../input/aman/web/Altered-Easy"
Medium_path = "../input/aman/web/Altered-Medium"
Hard_path = "../input/aman/web/Altered-Hard"


Easy_data = loading_data(Easy_path, train = True)
Medium_data = loading_data(Medium_path, train = True)
Hard_data = loading_data(Hard_path, train = True)
test = loading_data(Real_path, train = False)

data = np.concatenate([Easy_data, Medium_data, Hard_data], axis=0)

del Easy_data, Medium_data, Hard_data

Now let’s randomize the data and test the arrays to see what our data looks like:

In [ ]:
import random
random.shuffle(data)
random.shuffle(test)

Now I will split the image arrays and image labels:

In [ ]:
img, labels = [], []
for label, feature in data:
    labels.append(label)
    img.append(feature)
train_data = np.array(img).reshape(-1, img_size, img_size, 1)
train_data = train_data / 255.0

#### Building CNN for Gender Classification Model
Now, here I will build a Convolutional Neural Network for Gender classification model:

In [ ]:
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = Sequential([
                    Conv2D(32, 3, padding='same', activation='relu',kernel_initializer='he_uniform', input_shape = [96, 96, 1]),
                    MaxPooling2D(2),
                    Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation='relu'),
                    MaxPooling2D(2),
                    Flatten(),
                    Dense(128, kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(2, activation = 'softmax'),
                    ])

In [ ]:
model.summary()

Now, I will compile the model using Adam optimizers, with a learning rate of 10%, and to prevent our CNN model from overfitting, I will be using the early_stopping_call method:

In [ ]:
model.compile(optimizer = optimizers.Adam(1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
history = model.fit(train_data, train_labels, batch_size = 128, epochs = 30, 
          validation_split = 0.2, callbacks = [early_stopping_cb], verbose = 1)

Now, we have successfully trained our CNN for Gender Classification Model with a training accuracy of 99 per cent and Validation accuracy of 98 per cent. Now, let’s have a look at the performance of our Gender Classification Model:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)

As we split the training images into image labels and image arrays while training our model, we need to repeat the same process on the test images:

In [ ]:
test_images, test_labels = [], []

for label, feature in test:
    test_images.append(feature)
    test_labels.append(label)
    
test_images = np.array(test_images).reshape(-1, img_size, img_size, 1)
test_images = test_images / 255.0
del test
test_labels  = to_categorical(test_labels, num_classes = 2)

Now, let’s evaluate the performance of our CNN model on the test set:

In [ ]:
model.evaluate(test_images, test_labels)